In [ ]:
# Удаляем потенциально несовместимую версию numpy
!pip uninstall -y numpy

# Устанавливаем проверенную, стабильную версию numpy (1.23.5)
# и все остальные наши библиотеки
!pip install numpy==1.23.5
!pip install requests pandas gspread gspread-formatting google-auth-oauthlib google-auth-httplib2 pandas-ta

print("\n✅ Все библиотеки установлены. Если Colab предлагает перезапустить сессию (RESTART SESSION), пожалуйста, сделайте это.")

In [ ]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import pandas_ta as ta
from datetime import datetime
import traceback

# ... (Блоки 1 и 2 с функциями get_worksheet и calculate_indicators остаются без изменений) ...
# ... (привожу их для полноты картины, но менять их не нужно) ...

# ==============================================================================
# --- БЛОК 1: КОНФИГУРАЦИЯ И ПОДКЛЮЧЕНИЕ ---
# ==============================================================================
CREDS_FILE = 'credentials.json'
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1qBYS_DhGNsTo-Dnph3g_H27aHQOoY0EOcmCIKarb7Zc/"
SCOPE = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive.file']

def get_worksheet(sheet_name):
    try:
        creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPE)
        client = gspread.authorize(creds)
        spreadsheet = client.open_by_url(SPREADSHEET_URL)
        return spreadsheet.worksheet(sheet_name)
    except Exception as e:
        print(f"❌ Ошибка доступа к листу '{sheet_name}': {e}"); return None

# ==============================================================================
# --- БЛОК 2: ЯДРО АНАЛИТИКИ ---
# ==============================================================================
def calculate_indicators(df: pd.DataFrame) -> dict:
    if df.shape[0] < 50: return {}
    df.ta.rsi(length=14, append=True)
    df.ta.sma(length=20, append=True)
    df.ta.sma(length=50, append=True)
    df.ta.bbands(length=20, append=True)
    latest = df.iloc[-1]
    return {
        'RSI_14': round(latest.get('RSI_14'), 2) if pd.notna(latest.get('RSI_14')) else 'N/A',
        'MA_20': round(latest.get('SMA_20'), 2) if pd.notna(latest.get('SMA_20')) else 'N/A',
        'MA_50': round(latest.get('SMA_50'), 2) if pd.notna(latest.get('SMA_50')) else 'N/A',
        'BB_Upper': round(latest.get('BBU_20_2.0'), 2) if pd.notna(latest.get('BBU_20_2.0')) else 'N/A',
        'BB_Lower': round(latest.get('BBL_20_2.0'), 2) if pd.notna(latest.get('BBL_20_2.0')) else 'N/A',
    }

# ==============================================================================
# --- БЛОК 3: ГЛАВНАЯ ЛОГИКА АНАЛИЗАТОРА (ИСПРАВЛЕННАЯ) ---
# ==============================================================================
def main_analyzer():
    print("\n" + "="*50)
    print("--- 🧠 АСУП ИИ: Технический Анализатор v1.3 🧠 ---")
    print("="*50)

    sheets = {name: get_worksheet(name) for name in ['Holdings', 'History_OHLCV', 'Analysis']}
    if not all(sheets.values()): return

    print("🔄 Читаю список активов и исторические данные...")
    holdings_df = pd.DataFrame(sheets['Holdings'].get_all_records())
    history_df = pd.DataFrame(sheets['History_OHLCV'].get_all_records())

    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        history_df[col] = pd.to_numeric(history_df[col], errors='coerce')

    assets_to_analyze = holdings_df[
        (holdings_df['Type'].isin(['Stock_MOEX', 'Bond_MOEX'])) &
        (holdings_df['Watch'] == 'TRUE')
    ]

    print(f"☑️ Будет проанализировано {len(assets_to_analyze)} активов.")
    analysis_results = []

    for _, asset in assets_to_analyze.iterrows():
        ticker = asset['Ticker']
        print(f"  - Анализирую {ticker}...")
        ticker_history = history_df[(history_df['Ticker'] == ticker) & (history_df['Timeframe'] == 'D1')].copy()
        if ticker_history.empty: print(f"    - ⚠️ Нет данных."); continue

        ticker_history['Date'] = pd.to_datetime(ticker_history['Date'])
        ticker_history.sort_values(by='Date', inplace=True)

        indicators = calculate_indicators(ticker_history)
        if not indicators: print(f"    - ⚠️ Недостаточно данных для анализа (< 50 дней)."); continue

        # ИСПРАВЛЕНИЕ: Добавляем пустые значения для будущих столбцов
        analysis_results.append([
            ticker, datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            indicators.get('RSI_14'), indicators.get('MA_20'), indicators.get('MA_50'),
            indicators.get('BB_Upper'), indicators.get('BB_Lower'),
            "N/A", # Pattern_Found
            ""     # Alert_Sent (оставляем пустым)
        ])
        print(f"    - ✅ Анализ завершен. RSI_14: {indicators.get('RSI_14')}")

    if analysis_results:
        print("\n🔄 Записываю результаты в лист 'Analysis'...")
        try:
            # ИСПРАВЛЕНИЕ: Добавляем новый столбец в заголовки
            headers = ['Ticker', 'Last_Update', 'RSI_14', 'MA_20', 'MA_50', 'BB_Upper', 'BB_Lower', 'Pattern_Found', 'Alert_Sent']
            sheets['Analysis'].clear()
            sheets['Analysis'].update([headers] + analysis_results, value_input_option='USER_ENTERED')
            print("✅✅✅ УСПЕХ! Лист 'Analysis' обновлен с правильной структурой.")
        except Exception as e:
            print(f"❌ ОШИБКА при записи в 'Analysis': {e}"); traceback.print_exc()

    print("--- 🏁 РАБОТА АНАЛИЗАТОРА ЗАВЕРШЕНА 🏁 ---")

if __name__ == "__main__":
    main_analyzer()